In [30]:
import gym
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch_geometric.nn.conv import GCNConv
import math

Setup:
- Starting point: just try to train classifier on RL policies

In [31]:
### DQN implementation

# Define the Q-network
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

    def forward(self, x):
        return self.fc(x)
    
    def get_weights(self):
        return self.state_dict()

# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done

    def __len__(self):
        return len(self.buffer)

# DQN Agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=1e-2, batch_size=64, gamma=0.99, replay_size=1000):
        self.model = DQN(state_dim, action_dim, hidden_dim)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.replay_buffer = ReplayBuffer(replay_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.action_dim = action_dim
    
    def update(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        state, action, reward, next_state, done = self.replay_buffer.sample(self.batch_size)
        
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        if len(state.shape) == 1:
            state = state.reshape(-1, 1)
        if len(next_state.shape) == 1:
            next_state = next_state.reshape(-1, 1)
        action = torch.LongTensor(action)
        reward = torch.FloatTensor(reward)
        done = torch.FloatTensor(done)

        q_values = self.model.forward(state)
        next_q_values = self.model.forward(next_state)

        # state = state.T
        # next_state = next_state.T
        
        q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        next_q_value = next_q_values.max(1)[0]
        expected_q_value = reward + self.gamma * next_q_value * (1 - done)
        
        loss = nn.MSELoss()(q_value, expected_q_value.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def act(self, state, epsilon):
        if random.random() > epsilon:
            state = torch.FloatTensor(np.expand_dims(state, 0))
            q_value = self.model(state)
            action = q_value.max(-1)[1].item()
        else:
            action = random.randrange(self.action_dim)
        return action
    
class QTableAgent:
    def __init__(self, state_dim, action_dim, lr=1e-2, gamma=0.99):
        self.q_table = np.zeros((state_dim, action_dim))
        self.lr = lr
        self.gamma = gamma
        self.action_dim = action_dim
    
    def update(self, state, action, reward, next_state, done):
        q_value = self.q_table[state, action]
        next_q_value = np.max(self.q_table[next_state])
        expected_q_value = reward + self.gamma * next_q_value * (1 - done)
        
        self.q_table[state, action] += self.lr * (expected_q_value - q_value)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            action = np.argmax(self.q_table[state])
        else:
            action = random.randrange(self.action_dim)
        return action

In [205]:
NUM_NON_ZERO_REWARDS = 0
def one_hot_state(state, env):
    state_arr = np.zeros(env.observation_space.n)
    state_arr[state] = 1
    return state_arr

def train_dqn(env_name="CartPole-v1", episodes=500, epsilon_start=1.0, epsilon_final=0.01, 
              epsilon_decay=500, reward_function = None, verbose = False, return_reward = False, 
              print_every=50, **kwargs):
    """
    Train a DQN agent on the specified environment.
    
    Args:
        env_name: str
            Name of the environment to train the agent on.
        episodes: int
            Number of episodes to train the agent for.
        epsilon_start: float
            Initial epsilon value for epsilon-greedy action selection.
        epsilon_final: float
            Final epsilon value for epsilon-greedy action selection.
        epsilon_decay: float
            Decay rate for epsilon.
        reward_function: function
            Optional reward function to use for training.
        verbose: bool
            Whether to print training progress.

    Returns:
        DQNAgent: trained DQN agent. 
    """
    global NUM_NON_ZERO_REWARDS
    env = gym.make(env_name)
    if len(env.observation_space.shape) == 0:
        state_dim = env.observation_space.n
    else:
        state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = DQNAgent(state_dim, action_dim, **kwargs)
    
    epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * np.exp(-1. * frame_idx / epsilon_decay)
    
    rewards = np.zeros(episodes)   
    for episode in range(episodes):
        state = env.reset() # Reset the environment, reward
        if state_dim == env.observation_space.n:
            state = one_hot_state(state, env)
        episode_reward = 0
        while True:
            epsilon = epsilon_by_frame(episode)
            # One-hot encode the state
            action = agent.act(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            if state_dim == env.observation_space.n:
                next_state = one_hot_state(next_state, env)

            if reward_function and done: #custom reward function
                reward = reward_function(next_state)
            NUM_NON_ZERO_REWARDS += 0 if math.isclose(reward, 0) else 1
            
            agent.replay_buffer.push(state, action, reward, next_state, done)
            state = next_state
            episode_reward += reward
            
            agent.update()
            
            if done:
                break
            # print(f"Episode: {episode+1}, Total reward: {episode_reward}, Epsilon: {epsilon:.2f}")

        rewards[episode] = episode_reward
        # Optional: Render the environment to visualize training progress
        if verbose and episode % print_every == print_every - 1:
        #     render_env(env, agent)
            print(f"Episode: {episode+1}, Average total reward: {np.average(rewards[episode - print_every + 1 : episode])}, Epsilon: {epsilon:.2f}")

    env.close()
    return agent if not return_reward else (agent, rewards)

# Optional: Function to render the environment with the current policy
def render_env(env, agent):
    state = env.reset()
    done = False
    while not done:
        action = agent.act(state, 0)  # Using 0 epsilon for greedy action selection
        # print(env.step(action))
        next_state, reward, done, _ = env.step(action)
        env.render()
        state = next_state

In [206]:
def train_qtable(env_name="CartPole-v1", episodes=500, epsilon_start=1.0, epsilon_final=0.01, 
              epsilon_decay=500, reward_function = None, verbose = False, return_reward = False, 
              print_every=50, **kwargs):
    """
    Train a Q-table agent on the specified environment."""
    global NUM_NON_ZERO_REWARDS
    env = gym.make(env_name)
    if len(env.observation_space.shape) == 0:
        state_dim = env.observation_space.n
    else:
        state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = QTableAgent(state_dim, action_dim, **kwargs)

    rewards = np.zeros(episodes)
    epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * np.exp(-1. * frame_idx / epsilon_decay)
    for episode in range(episodes):
        state = env.reset()
        episode_reward = 0
        while True:
            epsilon = epsilon_by_frame(episode)
            action = agent.act(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            if reward_function:
                reward = reward_function(next_state)

            agent.update(state, action, reward, next_state, done)
            state = next_state
            episode_reward += reward
            if done:
                break

        rewards[episode] = episode_reward
        if verbose and episode % print_every == print_every - 1:
            print(f"Episode: {episode+1}, Average total reward: {np.average(rewards[episode - print_every + 1 : episode])}, Epsilon: {epsilon:.2f}")
        
    env.close()
    return agent if not return_reward else (agent, rewards)
    

In [129]:
NEAR_ZERO = 1e-9
def test_dqn(env, agent, episodes=10, reward_function=None, verbose = False):
    print(f"Maximum reward: {env.spec.reward_threshold}")
    average_value = 0
    for episode in range(episodes):
        # if episode == 0:
        #     render_env(env, agent)
        state = env.reset()
        if len(env.observation_space.shape) == 0:
            state = one_hot_state(state, env)
        episode_reward = 0
        done = False
        while not done:
            action = agent.act(state, 0)  # Using 0 epsilon for greedy action selection
            next_state, reward, done, _ = env.step(action)
            if len(env.observation_space.shape) == 0:
                next_state = one_hot_state(next_state, env)
            if reward_function and done:
                reward = reward_function(next_state)
            episode_reward += reward
            state = next_state
        if verbose:
            print(f"Episode: {episode+1}, Total reward: {episode_reward}")
        average_value += episode_reward
    average_value /= episodes
    print(f"Average reward: {average_value}")
    

In [130]:
def test_qtable(env, agent, episodes=10, reward_function=None, verbose = False):
    """
    Test a Q-table agent on the specified environment.
    (This is basically test_dqn but without the one-hot encoding.)
    """
    print(f"Maximum reward: {env.spec.reward_threshold}")
    average_value = 0
    for episode in range(episodes):
        state = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action = agent.act(state, 0)  # Using 0 epsilon for greedy action selection
            next_state, reward, done, _ = env.step(action)
            if reward_function:
                reward = reward_function(next_state)
            episode_reward += reward
            state = next_state
        if verbose:
            print(f"Episode: {episode+1}, Total reward: {episode_reward}")
        average_value += episode_reward
    average_value /= episodes
    print(f"Average reward: {average_value}")

In [121]:
env_name = "Taxi-v3"
agent, rewards = train_qtable(env_name = env_name, episodes = 10000, verbose = True, return_reward = True,
                           epsilon_decay=10, lr=0.1, gamma=0.9)
# rewards

Episode: 50, Total reward: -151, Epsilon: 0.02
Episode: 100, Total reward: -209, Epsilon: 0.01
Episode: 150, Total reward: -184, Epsilon: 0.01
Episode: 200, Total reward: -235, Epsilon: 0.01
Episode: 250, Total reward: -285, Epsilon: 0.01
Episode: 300, Total reward: -293, Epsilon: 0.01
Episode: 350, Total reward: -362, Epsilon: 0.01
Episode: 400, Total reward: -32, Epsilon: 0.01
Episode: 450, Total reward: 14, Epsilon: 0.01
Episode: 500, Total reward: 13, Epsilon: 0.01
Episode: 550, Total reward: 13, Epsilon: 0.01
Episode: 600, Total reward: -38, Epsilon: 0.01
Episode: 650, Total reward: -59, Epsilon: 0.01
Episode: 700, Total reward: -28, Epsilon: 0.01
Episode: 750, Total reward: -27, Epsilon: 0.01
Episode: 800, Total reward: -90, Epsilon: 0.01
Episode: 850, Total reward: 12, Epsilon: 0.01
Episode: 900, Total reward: 10, Epsilon: 0.01
Episode: 950, Total reward: 10, Epsilon: 0.01
Episode: 1000, Total reward: 7, Epsilon: 0.01
Episode: 1050, Total reward: -19, Epsilon: 0.01
Episode: 1100

In [26]:
test_qtable(gym.make(env_name), agent, episodes = 100)
# agent.model.get_weights()

Maximum reward: 8
Average reward: 7.87


In [36]:
### Coherence classifier

#agent.model.get_weights()

# Define a simple GCN model
from torch_geometric.data import Data
class GCN(torch.nn.Module):
    def __init__(self, data):
        super(GCN, self).__init__()
        # Define the GCN layers
        self.conv1 = GCNConv(data.num_node_features, 4)  # Input features to hidden
        self.conv2 = GCNConv(4, 2)  # Hidden to output features
        self.data = data

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Pass data through the first GCN layer, then apply ReLU
        x = torch.relu(self.conv1(x, edge_index))
        # Pass data through the second GCN layer
        x = self.conv2(x, edge_index)
        return x
    
    
def nn_to_data(model: nn.Module) -> Data:
    edges = []

    # Counter for global neuron index
    idx = 0

    # Iterate over each layer in the network
    base = next(model.children())
    if isinstance(base, nn.Sequential):
        layers = list(base.children())
        layers2 = list(base.children())
    else:
        layers = list(model.children()) # iterator over the layers of the model
        layers2 = list(model.children())
    
    num_nodes = layers2[0].weight.shape[1] + sum([layer.weight.shape[0] for layer in layers2 if isinstance(layer, nn.Linear)])
    num_node_features = num_nodes
    node_features = torch.zeros(num_nodes, num_node_features)
    # shape = (num_nodes, num_node_features), where the node features are the bias of each node
    # and the weights of the edges to each node (zero if there is no edge)

    for layer in layers:
        if isinstance(layer, nn.Linear):
            # Update edges based on the weight matrix
            input_dim = layer.weight.shape[1]
            output_dim = layer.weight.shape[0]
            for i in range(input_dim):  # Input neurons (e.g. 4)
                for j in range(output_dim):  # Output neurons (e.g. 64)
                    edges.append((idx + i, idx + input_dim + j))
            
            # Update node features (e.g., biases)
            biases = torch.tensor(layer.bias.detach().numpy())
            edge_weights = torch.tensor(layer.weight.detach().numpy().T)
            node_features[idx + input_dim:idx + input_dim + output_dim, 0] = biases
            node_features[idx:idx + input_dim, 1+idx:1+idx+output_dim] = edge_weights
            node_features[idx + input_dim:idx + input_dim + output_dim, 1+idx:1+idx+input_dim] = edge_weights.T
            
            # Update the global neuron index
            idx += input_dim

    # Convert lists to PyTorch tensors
    num_nonzero = [np.count_nonzero(node_features[i]) for i in range(node_features.shape[0])]
    # print(num_nonzero)
    row_mean, row_median, row_var = torch.mean(node_features[:, 1:], dim=1), torch.median(node_features[:, 1:], dim=1)[0], torch.var(node_features[:, 1:], dim=1)
    x = torch.stack([node_features[:, 0], row_mean, row_median, row_var]).T
    # print(x.shape)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return Data(x=x, edge_index=edge_index)

data = nn_to_data(agent.model)
gcn = GCN(data)
# data.x.shape, data.edge_index.shape
# print(data.x)

#Debug
out_of_bounds = data.edge_index >= data.x.shape[0]
if out_of_bounds.any():
    print("Out-of-bounds indices found at locations:")
    print(data.edge_index[:, out_of_bounds.any(dim=0)])

AttributeError: 'QTableAgent' object has no attribute 'model'

In [42]:
# Dataset generation
env = gym.make(env_name)
NEAR_ZERO = 1e-9
NUM_REWARD_CALLS = 0
NUM_NON_ZERO_REWARDS = 0
def deterministic_random(*args, lb = -1, ub = 1, sparsity = 0.0, continuous = False):
    """
    Create a deterministic random number generator for a given set of arguments.
    Used to generate deterministic reward functions for the coherence classifier.
    [Edit 4/3/24: adapted to continuous state space]"""
    global NUM_REWARD_CALLS
    NUM_REWARD_CALLS += 1
    unique_seed = f"{args}".encode("utf-8")
    random.seed(unique_seed)
    return random.uniform(lb, ub) if random.random() > sparsity else random.uniform(-NEAR_ZERO, NEAR_ZERO)

NUM_TRAIN_R_FUNCS = 1
NUM_EPS_TRAIN_R = 50
URS_r_funcs = [lambda *args: deterministic_random(args) for _ in range(NUM_TRAIN_R_FUNCS)]
URS_agents = [train_dqn(env_name = env_name, 
                        episodes=NUM_EPS_TRAIN_R, reward_function=r_func) for r_func in URS_r_funcs]
USS_r_funcs = [lambda *args: deterministic_random(args, sparsity=0.999) for _ in range(NUM_TRAIN_R_FUNCS)]
print(f"Number of reward function calls: {NUM_REWARD_CALLS}")
print(f"Number of non-zero rewards: {NUM_NON_ZERO_REWARDS}")
USS_agents = [train_dqn(env_name = env_name, 
                        episodes=NUM_EPS_TRAIN_R, reward_function=r_func) for r_func in USS_r_funcs]

Number of reward function calls: 50
Number of non-zero rewards: 9871


In [9]:
# Test if deterministic_random is deterministic and has the correct sparsity
assert deterministic_random(1, 2, 3, 4) == deterministic_random(1, 2, 3, 4)
assert not deterministic_random(1, 2, 3, 4) == deterministic_random(1, 2, 3, 6)
[deterministic_random(1, 2, 3, i, sparsity = 0.5) for i in range(10)]

[0.6014137224608205,
 5.734868810378968e-10,
 0.18947200717913826,
 -0.11464719428521586,
 1.9375194864306798e-11,
 3.1131227593489704e-10,
 -7.023178277046693e-10,
 2.965355797951794e-10,
 0.41831271768541045,
 -3.207247699354683e-10]

In [13]:
# Test when do USS agents have non-zero rewards
USS_test_r_func = lambda *args: deterministic_random(args, sparsity=0.0)
assert USS_test_r_func(42) == USS_test_r_func(42)
USS_test_agent = train_dqn(env_name = env_name, episodes=500, reward_function=USS_test_r_func, 
                           verbose = True)
# Epsilon measuring how much the agent is exploring

Episode: 50, Total reward: -3.423544244882467, Epsilon: 0.91
Episode: 100, Total reward: -2.9740031925320296, Epsilon: 0.82
Episode: 150, Total reward: 0.44985805334502116, Epsilon: 0.74
Episode: 200, Total reward: 1.9841901190163609, Epsilon: 0.67
Episode: 250, Total reward: 1.9401242078162357, Epsilon: 0.61
Episode: 300, Total reward: -1.4982150868261042, Epsilon: 0.55
Episode: 350, Total reward: -3.697850242769119, Epsilon: 0.50
Episode: 400, Total reward: 2.4217055508022867, Epsilon: 0.46
Episode: 450, Total reward: -3.5300506186568104, Epsilon: 0.41
Episode: 500, Total reward: -1.2907808496919664, Epsilon: 0.37


In [11]:
# epsilon_final, epsilon_start, epsilon_decay = 0.01, 1.0, 500
# [epsilon_final + (epsilon_start - epsilon_final) * np.exp(-1. * frame_idx / epsilon_decay) for frame_idx in range(500)]
test_dqn(gym.make("CartPole-v1"), USS_test_agent, reward_function=USS_test_r_func)

Maximum reward: 475.0
Episode: 1, Total reward: -15.55250607481602
Episode: 2, Total reward: 7.210228177137941
Episode: 3, Total reward: -8.919077872422985
Episode: 4, Total reward: 3.3986894664967524
Episode: 5, Total reward: -5.9617775051492
Episode: 6, Total reward: -8.334476064077458
Episode: 7, Total reward: 0.05630472186067914
Episode: 8, Total reward: -2.9496726002074958
Episode: 9, Total reward: -17.47453520502488
Episode: 10, Total reward: 27.669555440546127


In [46]:
def get_state_shape(env):
    return 1 if len(env.observation_space.shape) == 0 else env.observation_space.shape[0]
def get_state_size(env):
    return env.observation_space.n if len(env.observation_space.shape) == 0 else env.observation_space.shape[0]
UPS_agents = [DQNAgent(get_state_size(env), env.action_space.n) for _ in range(NUM_TRAIN_R_FUNCS)]

import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool, GCNConv, GATConv

class GraphLevelGCN(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GraphLevelGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.linear = torch.nn.Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # edge_weights = data.edge_attr
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        
        # Aggregate node features to graph-level features
        x = global_mean_pool(x, batch)
        
        # Make a binary classification prediction
        x = self.linear(x)
        return torch.sigmoid(x)

class GATGraphLevelBinary(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GATGraphLevelBinary, self).__init__()
        self.conv1 = GATConv(num_node_features, 8, heads=8, dropout=0.6)
        # Increase the number of output features from the first GAT layer
        self.conv2 = GATConv(8 * 8, 16, heads=1, concat=False, dropout=0.6)
        # Additional GAT layer for richer node representations
        self.linear = torch.nn.Linear(16, 1)
        # Final linear layer to produce a graph-level output

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        x = global_mean_pool(x, batch)  # Aggregate node features to graph-level
        x = self.linear(x)
        return torch.sigmoid(x)  # Sigmoid activation function for binary classification

# Training loop
USS_data = [nn_to_data(agent.model) for agent in USS_agents]
URS_data = [nn_to_data(agent.model) for agent in URS_agents]
print(URS_data[0].x.shape)
UPS_data = [nn_to_data(agent.model) for agent in UPS_agents]
assert URS_data[0].x.shape == UPS_data[0].x.shape

torch.Size([634, 4])


In [49]:
# Binary classification between two datasets
dataset1 = URS_data
dataset2 = UPS_data
def generate_data(dataset1, dataset2):
    indices = np.random.permutation(len(dataset1) + len(dataset2))
    data = [dataset1[i] if i < len(dataset1) else dataset2[i - len(dataset1)] for i in indices]
    for i in range(len(data)):
        data[i].y = 1.0 if indices[i] < len(dataset1) else 0.0 # Binary labels for each node; 1 = URS, 0 = UPS
        # Hence roughly speaking, 1 = more coherent, 0 = less coherent

    train_data_ratio = 0.8
    train_data, test_data = data[:int(train_data_ratio * len(data))], data[int(train_data_ratio * len(data)):]
    num_node_features = data[0].x.shape[1] # Number of features for each node
    return train_data, test_data, num_node_features

train_data, test_data, num_node_features = generate_data(dataset1, dataset2)
# Loss and optimizer
model = GraphLevelGCN(num_node_features)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

def train_classifier(model, criterion, optimizer, train_data, test_data, epochs = 40, patience = 3, 
                     epochs_without_improvement = 0, best_loss = float('inf')):
    for epoch in range(epochs):
        avg_train_loss = 0
        for datapt in train_data:
            model.train()
            optimizer.zero_grad()

            # print(f"datapt.x shape: {datapt.x.shape}")  # Should be [num_nodes, num_node_features]
            # print(f"datapt.edge_index shape: {datapt.edge_index.shape}")  # Should be [2, num_edges]
            out = model.forward(datapt)
            # print(out.size())
            # print(torch.tensor([[datapt.y]]).size())
            loss = criterion(out, torch.tensor([[datapt.y]]))  # Adjust shape as necessary
            loss.backward()
            optimizer.step()
            avg_train_loss += loss.item()
        avg_train_loss /= len(train_data)

        avg_test_loss = 0
        for datapt in test_data:
            model.eval()
            with torch.no_grad():
                out = model.forward(datapt)
                loss = criterion(out, torch.tensor([[datapt.y]]))
                avg_test_loss += loss.item()
        avg_test_loss /= len(test_data)
        
        print(f'Epoch {epoch+1}: Average Train Loss: {avg_train_loss}, Average Test Loss: {avg_test_loss}')
        
        # Early Stopping
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break

train_classifier(model, criterion, optimizer, train_data, test_data)

Epoch 1: Average Train Loss: 0.611354649066925, Average Test Loss: 0.8701407313346863
Epoch 2: Average Train Loss: 0.583457887172699, Average Test Loss: 0.9334022402763367
Epoch 3: Average Train Loss: 0.5527416467666626, Average Test Loss: 1.0072805881500244
Epoch 4: Average Train Loss: 0.5177010297775269, Average Test Loss: 1.0963069200515747
Early stopping at epoch 4


In [37]:
# Test GCN model on a "more powerful" NN
print(model.forward(dataset1[0]))
print(model.forward(dataset2[0]))
powerful_models = [nn_to_data(train_dqn(env_name = env_name, episodes = 5 * i).model) 
                   for i in [1, 3, 10]]
print([model.forward(data) for data in powerful_models])

tensor([[1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.0007]], grad_fn=<SigmoidBackward0>)
[tensor([[0.2728]], grad_fn=<SigmoidBackward0>), tensor([[0.9962]], grad_fn=<SigmoidBackward0>), tensor([[1.]], grad_fn=<SigmoidBackward0>)]


- The classifier training process is finicky -- sometimes it overfits, sometimes it underfits -- but sometimes can reach very low loss (< 0.002)
- Even weak classifiers classify powerful models (a.k.a. agents with >15 episodes in CartPole) as having P(URS) = 1, corresponding to coherence ~ $\infty$
- P(USS) / P(URS) is still having trouble as a metric; seems extremely difficult to detect differences between USS and URS-generated policies here with current methods
    - We will need some kind of "more advanced" coherence metric to distinguish more advanced policies; TODO: implement UUS somehow
- Adding node weights to every other node to the features passed into the GCN (such that, in CartPole, the data matrix has shape (134, 134) instead of (134, 1)) makes the GCN much worse, probably because of higher dimensionality
    - Using attention in the GNN does not help, and in fact actively overfits when using (134, 1) data
- Even with sparsity = 0.999, USS is still hard to distinguish
- For simpler discrete environments, maybe a Q-table is enough to solve the problem
- Takes >= 500 episodes, small epsilon to effectively learn DQN policy


In [72]:
# Now classifying q-table agents
NUM_EPS_TRAIN_R = 1000
NUM_TRAIN_R_FUNCS = 50
NUM_REWARD_CALLS = 0
env = gym.make(env_name)
def deterministic_random(*args, lb = -1, ub = 1, sparsity = 0.0, continuous = False):
    """
    Create a deterministic random number generator for a given set of arguments.
    Used to generate deterministic reward functions for the coherence classifier.
    [Edit 4/3/24: adapted to continuous state space]"""
    global NUM_REWARD_CALLS
    NUM_REWARD_CALLS += 1
    unique_seed = f"{args}".encode("utf-8")
    random.seed(unique_seed)
    return random.uniform(lb, ub) if random.random() > sparsity else random.uniform(-NEAR_ZERO, NEAR_ZERO)
URS_r_funcs = [lambda *args: deterministic_random(args) for _ in range(NUM_TRAIN_R_FUNCS)]
URS_agents = [train_qtable(env_name = env_name, episodes=NUM_EPS_TRAIN_R, 
                           reward_function = r_func) for r_func in URS_r_funcs]
USS_r_funcs = [lambda *args: deterministic_random(args, sparsity=0.99) for _ in range(NUM_TRAIN_R_FUNCS)]
USS_agents = [train_qtable(env_name = env_name, episodes=NUM_EPS_TRAIN_R,
                            reward_function = r_func) for r_func in USS_r_funcs]
UPS_agents = [QTableAgent(get_state_size(env), env.action_space.n) for _ in range(NUM_TRAIN_R_FUNCS)]

# The Q-Table is already one-hot encoded, so we don't need to convert it to a Data object

In [210]:
# Test ground
print([USS_r_funcs[0](i) for i in range(10)])
test_USS_agent = train_qtable(env_name = env_name, episodes = 50, verbose=True, epsilon_decay = 100, 
                              lr = 0.01, gamma = 0.9, reward_function = USS_r_funcs[0])
test_qtable(gym.make(env_name), test_USS_agent, episodes = 100, reward_function = USS_r_funcs[0])
test_UPS_agent = QTableAgent(get_state_size(env), env.action_space.n)
test_qtable(gym.make(env_name), test_UPS_agent, episodes = 100, reward_function = USS_r_funcs[0])

[6.879610979477867e-10, -8.537837971656553e-10, 1.3453720557822044e-10, -6.91582481382308e-10, 5.834328244211258e-10, 2.689469001408139e-10, -7.747733415111116e-10, 3.5242825351422126e-10, 2.6150563760737437e-10, -8.919200340033589e-10]
Episode: 50, Average total reward: 5.595625995452984, Epsilon: 0.62
Maximum reward: 8
Average reward: 2.7499838254254803
Maximum reward: 8
Average reward: 0.0067166083132540865


In [158]:
class FCNNBinary(nn.Module):
    def __init__(self, num_node_features):
        super(FCNNBinary, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_node_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, data):
        x = data.x
        x = self.fc(x)
        return torch.sigmoid(x)
    
def qtable_to_feat(qtable: torch.Tensor, label):
    # In qtable, rows are states and columns are actions taken in that state
    return Data(x = torch.flatten(qtable), y = label) # Naive approach

In [185]:
UPS_agents = [QTableAgent(get_state_size(env), env.action_space.n) for _ in range(NUM_TRAIN_R_FUNCS)]

for agent in UPS_agents:
    for row in agent.q_table:
        for i in range(len(row)):
            row[i] = np.random.uniform(-1, 1) # set each value to a random number between -1 and 1
dataset1 = [qtable_to_feat(torch.tensor(agent.q_table, dtype=torch.float32), 1) for agent in USS_agents]
dataset2 = [qtable_to_feat(torch.tensor(agent.q_table, dtype=torch.float32), 0) for agent in URS_agents] # URS = 1, UPS = 0

def generate_fcnn_data(dataset1, dataset2):
    indices = np.random.permutation(len(dataset1) + len(dataset2))
    data = [dataset1[i] if i < len(dataset1) else dataset2[i - len(dataset1)] for i in indices]
    for i in range(len(data)):
        data[i].y = 1.0 if indices[i] < len(dataset1) else 0.0 # Binary labels for each node; 1 = URS, 0 = UPS
        # Hence roughly speaking, 1 = more coherent, 0 = less coherent

    train_data_ratio = 0.8
    train_data, test_data = data[:int(train_data_ratio * len(data))], data[int(train_data_ratio * len(data)):]
    num_node_features = data[0].x.shape[0] # Number of features for each node
    return train_data, test_data, num_node_features

def train_fcnn_classifier(model, criterion, optimizer, train_data, test_data, epochs = 40, patience = 3, 
                          epochs_without_improvement = 0, best_loss = float('inf')):
    for epoch in range(epochs):
        avg_train_loss = 0
        for datapt in train_data:
            model.train()
            optimizer.zero_grad()

            out = model.forward(datapt)
            assert isinstance(out, torch.Tensor), f"Expected model.forward to return a tensor, but got {out}"
            loss = criterion(out, torch.tensor([datapt.y]))  # Adjust shape as necessary
            loss.backward()
            optimizer.step()
            avg_train_loss += loss.item()
        avg_train_loss /= len(train_data)

        avg_test_loss = 0
        for datapt in test_data:
            model.eval()
            with torch.no_grad():
                out = model.forward(datapt)
                loss = criterion(out, torch.tensor([datapt.y]))
                avg_test_loss += loss.item()
        avg_test_loss /= len(test_data)
        
        print(f'Epoch {epoch+1}: Average Train Loss: {avg_train_loss}, Average Test Loss: {avg_test_loss}')
        
        # Early Stopping
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break

train_data, test_data, num_node_features = generate_fcnn_data(dataset1, dataset2)
print(num_node_features)
model = FCNNBinary(num_node_features)
criterion = torch.nn.BCELoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
train_fcnn_classifier(model, criterion, optimizer, train_data, test_data)

3000
Epoch 1: Average Train Loss: 45.84827167169324, Average Test Loss: 65.0
Epoch 2: Average Train Loss: 46.25, Average Test Loss: 65.0
Epoch 3: Average Train Loss: 46.25, Average Test Loss: 65.0
Epoch 4: Average Train Loss: 46.25, Average Test Loss: 65.0
Early stopping at epoch 4


In [ ]:
def generate_UVS_qagent(rand_qtable):
    """
    Generate a Q-table agent with """

In [186]:
### Turn the state and action space of Taxi-v3 into a graph

from collections import defaultdict
taxi_env = gym.make("Taxi-v3")
taxi_env.reset()
# Initialize containers for graph data
edges = defaultdict(list)
edge_attr = defaultdict(list)

# A helper function to encode the state into a single number (node index)
def state_to_node(taxi_row, taxi_col, pass_loc, dest_idx):
    # This encoding assumes specific knowledge about the Taxi-v3 state space size
    return taxi_row * 100 + taxi_col * 20 + pass_loc * 4 + dest_idx
    # max = 4 * 100 + 4 * 20 + 4 * 4 + 3 = 400 + 80 + 16 + 3 = 499

# Iterate through all possible states and actions to construct the graph
for taxi_row in range(5):
    for taxi_col in range(5):
        for pass_loc in range(5):  # 4 locations + 1 for 'in taxi'
            for dest_idx in range(4):
                current_state = state_to_node(taxi_row, taxi_col, pass_loc, dest_idx)
                for action in range(taxi_env.action_space.n):
                    # Set the environment to the current state
                    taxi_env.unwrapped.s = current_state
                    # Take action and observe the next state and reward
                    next_state, reward, done, _ = taxi_env.step(action)
                    # Add edge from current state to next state
                    edges[current_state].append(next_state)
                    # Optionally, use rewards as edge attributes
                    # edge_attr[(current_state, next_state)].append(reward)
                    taxi_env.reset()


# Convert edges and edge attributes to tensors
edge_index = []
for src, dsts in edges.items():
    for dst in dsts:
        edge_index.append([src, dst])
edge_index = torch.tensor(edge_index).t().contiguous()
edge_index, edge_index.shape

(tensor([[  0,   0,   0,  ..., 499, 499, 499],
         [100,   0,  20,  ..., 479, 499, 499]]),
 torch.Size([2, 3000]))

In [219]:
def greedy_policy(q_table):
    return torch.tensor(np.argmax(q_table, axis=1).reshape(-1, 1).astype(np.float32))
def random_policy(state_dim):
    return torch.randint(0, 6, (state_dim, 1)).float()

dataset1 = [Data(x = greedy_policy(agent.q_table), edge_index = edge_index, y = 1) for agent in URS_agents]
# dataset2 = [Data(x = greedy_policy(agent.q_table), edge_index = edge_index, y = 0) for agent in URS_agents]
dataset2 = [Data(x = random_policy(agent.q_table.shape[0]), edge_index = edge_index, y = 0) for agent in UPS_agents]
print(dataset1[0].x.shape)

train_data, test_data, num_node_features = generate_data(dataset1, dataset2)
model = GraphLevelGCN(num_node_features)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
train_classifier(model, criterion, optimizer, train_data, test_data)

torch.Size([500, 1])


Epoch 1: Average Train Loss: 0.7374219067802187, Average Test Loss: 0.5225726202130317
Epoch 2: Average Train Loss: 0.38468063431232624, Average Test Loss: 0.033242557127960023
Epoch 3: Average Train Loss: 0.008957611854202695, Average Test Loss: 0.003896899623214267
Epoch 4: Average Train Loss: 0.003252486547717126, Average Test Loss: 0.002658354368759319
Epoch 5: Average Train Loss: 0.002364098767429823, Average Test Loss: 0.0020714492442493793
Epoch 6: Average Train Loss: 0.0019292355794277682, Average Test Loss: 0.0017695214715786277
Epoch 7: Average Train Loss: 0.0016966756207693833, Average Test Loss: 0.0016076479536422995
Epoch 8: Average Train Loss: 0.0015676271767006255, Average Test Loss: 0.001522450581251178
Epoch 9: Average Train Loss: 0.001495974242243392, Average Test Loss: 0.0014816406735917553
Epoch 10: Average Train Loss: 0.0014567843648819689, Average Test Loss: 0.0014669116084405688
Epoch 11: Average Train Loss: 0.001435726875297405, Average Test Loss: 0.001469265490

In [198]:
print(torch.transpose(dataset1[0].x, 0, -1)[:, 0:50]) # Example greedy policies
print(torch.transpose(dataset2[0].x, 0, -1)[:, 0:50])
print(model.forward(dataset1[0]))
print(model.forward(dataset2[0]))

tensor([[0., 2., 1., 1., 1., 0., 1., 2., 3., 1., 0., 3., 1., 1., 3., 0., 5., 0.,
         3., 3., 0., 1., 3., 3., 0., 0., 2., 3., 3., 1., 0., 3., 0., 0., 3., 0.,
         2., 1., 3., 2., 0., 3., 3., 3., 1., 0., 1., 1., 3., 1.]])
tensor([[5., 0., 0., 4., 3., 0., 2., 0., 1., 1., 2., 4., 5., 1., 4., 5., 1., 0.,
         2., 5., 3., 4., 0., 3., 4., 4., 4., 0., 4., 3., 4., 5., 3., 4., 2., 0.,
         1., 4., 2., 5., 4., 0., 4., 2., 5., 0., 1., 0., 3., 4.]])
tensor([[0.9999]], grad_fn=<SigmoidBackward0>)
tensor([[0.0076]], grad_fn=<SigmoidBackward0>)


In [199]:
taxi_model = train_qtable(env_name = "Taxi-v3", episodes = 3000, verbose = True, print_every = 1000, 
                          return_reward = False)
taxi_data = Data(x = torch.tensor(greedy_policy(taxi_model.q_table)), edge_index = edge_index)
print(model.forward(taxi_data))

Episode: 1000, Total reward: -299, Epsilon: 0.14
Episode: 2000, Total reward: -112, Epsilon: 0.03
Episode: 3000, Total reward: -200, Epsilon: 0.01
tensor([[0.9997]], grad_fn=<SigmoidBackward0>)
